In [16]:
import time
import pandas as pd
import datetime as dt

from lib.utils import save_symbol_fullbook_data

_dt_dt_ = dt.datetime
_dt_td_ = dt.timedelta
_dt_d_  = dt.date

from binance.client import Client

pd.options.display.float_format = '{:,.10f}'.format

In [2]:
api_key_info = pd.read_csv('/home/yizhang/Research/api_key.csv')
api_key      = api_key_info.loc[0, 'value']
api_secret   = api_key_info.loc[1, 'value']

client = Client(api_key, api_secret)

In [3]:
def clean_trade_data(trades):
    trades[['price', 'qty', 'time', 'bid', 'ask']] = trades[['price', 'qty', 'time', 'bid', 'ask']].astype(float)
    trades.loc[:, 'sign'] = 0
    trades.loc[:, 'mid']  = 0.5 * (trades.bid + trades.ask)
    trades.loc[trades.price < trades.mid, 'sign'] = -1
    trades.loc[trades.price > trades.mid, 'sign'] = 1 

In [4]:
# get market depth
symbol    = 'IOSTETH'
delay     = 1
iteration = 0

MAX_ITERATIONS = 10

fullbook = pd.DataFrame()

while(iteration < MAX_ITERATIONS):
    iteration += 1
    
    depth   = pd.DataFrame(client.get_order_book(symbol=symbol))
    
    book_update = pd.DataFrame({_dt_dt_.now().strftime("%Y%m%d %H:%M:%S.%f"): {'q_bid': float(depth.loc[0, 'bids'][1]), 'p_bid': float(depth.loc[0, 'bids'][0]), 
                               'q_ask': float(depth.loc[0, 'asks'][1]), 'p_ask': float(depth.loc[0, 'asks'][0]), 
                               'q_bid_2': float(depth.loc[1, 'bids'][1]), 'p_bid_2': float(depth.loc[1, 'bids'][0]), 
                               'q_ask_2': float(depth.loc[1, 'asks'][1]), 'p_ask_2': float(depth.loc[1, 'asks'][0])
                              }})
    
    fullbook = fullbook.append(book_update.T)
    
    time.sleep(delay)

In [13]:
def get_market_data(client, symbol, end_time, delay=1):
    fullbook = pd.DataFrame()
    while(_dt_dt_.now() < end_time):
        depth   = pd.DataFrame(client.get_order_book(symbol=symbol))

        book_update = pd.DataFrame({_dt_dt_.now().strftime("%Y%m%d %H:%M:%S.%f"): {'q_bid': float(depth.loc[0, 'bids'][1]), 'p_bid': float(depth.loc[0, 'bids'][0]), 
                                   'q_ask': float(depth.loc[0, 'asks'][1]), 'p_ask': float(depth.loc[0, 'asks'][0]), 
                                   'q_bid_2': float(depth.loc[1, 'bids'][1]), 'p_bid_2': float(depth.loc[1, 'bids'][0]), 
                                   'q_ask_2': float(depth.loc[1, 'asks'][1]), 'p_ask_2': float(depth.loc[1, 'asks'][0])
                                   }})

        fullbook = fullbook.append(book_update.T)

        time.sleep(delay)

    save_symbol_fullbook_data(symbol, fullbook, '/home/yizhang/Research/data/%s' % (_dt_dt_.now().strftime("%Y/%m/%d/")))

In [23]:
ticker_list = [x['symbol'] for x in client.get_all_tickers()]
for symbol in ticker_list:
    get_market_data(client, symbol, end_time = _dt_dt_.now() + _dt_td_(seconds=5))

ValueError: arrays must all be same length

In [25]:
_dt_d_.today()

datetime.date(2018, 6, 6)

In [ ]:
_dt_dt_.combine()